## Data Preprocessing

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from scipy.misc import imread, imsave, imresize
from natsort import natsorted

path = 'E:/dataSets/chars74K/data'

# preprocessed image size
img_height, img_width = 32, 32

# preprocessed image dir
suffix = 'Preproc'
train_data_path = path + '/train' + suffix
test_data_path = path + '/test' + suffix

# create dir
if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)

if not os.path.exists(test_data_path):
    os.makedirs(test_data_path)
    
# process image's size and color
for datasetType in ['train', 'test']:
    imgFiles = natsorted(glob.glob(path + '/' + datasetType + '/*'))
    imgData = np.zeros((len(imgFiles), img_height, img_width))
    
    for i, imgFilePath in enumerate(imgFiles):
        # Color image
        img = imread(imgFilePath, True)
        
        imgResized = imresize(img, (img_height, img_width))
        imgData[i] = imgResized
        
        # store processed image
        filename = os.path.basename(imgFilePath)
        filenameDotSplit = filename.split('.')
        newFilename = str(int(filenameDotSplit[0])).zfill(5) + "." + str(filenameDotSplit[-1].lower())
        newFilePath = path + '/' + datasetType + suffix + "/" + newFilename
        imsave(newFilePath, imgResized)
        
    print('origin img data shape: ', imgData.shape)
    imgData = imgData[:, :, :, np.newaxis]
    print('new img data shape: ', imgData.shape)
        
    imgData = imgData.astype('float32') / 255.
    np.save(path + '/' + datasetType + suffix + '.npy', imgData)

D:\anaconda\envs\keras-gpu\lib\site-packages\scipy\misc\pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if issubdtype(ts, int):
D:\anaconda\envs\keras-gpu\lib\site-packages\scipy\misc\pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


origin img data shape:  (6283, 32, 32)
new img data shape:  (6283, 32, 32, 1)


origin img data shape:  (6220, 32, 32)
new img data shape:  (6220, 32, 32, 1)


### Encode Label

In [12]:
import keras

def label2int(ch):
    #process into 0 - 61
    asciiVal = ord(ch)
    if (asciiVal <= 57): # 0-9
        asciiVal -= 48
    elif (asciiVal <= 90): #A -Z
        asciiVal -= 55
    else: # a-z
        asciiVal -= 61
    return asciiVal

def int2label(i):
    if (i <= 9):
        i += 48
    elif (i <= 35):
        i += 55
    else:
        i += 61
    return chr(i)

path = 'E:/dataSets/chars74K/data'

# read label
y_train = pd.read_csv(path + '/trainLabels.csv').values[:, 1]

# Encode label
Y_train = np.zeros((y_train.shape[0], 62))
for i in range(y_train.shape[0]):
    Y_train[i][label2int(y_train[i])] = 1 # One-hot
    
np.save(path + '/' + 'labelsPreproc.npy', Y_train)

### CNN Model

In [14]:
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

batch_size = 128
nb_classes = 62
nb_epoch = 10
img_height, img_width = 32, 32
path = 'E:/dataSets/chars74K/data'

# Load preprocessed data and label
X_train_all = np.load(path + '/trainPreproc.npy')
Y_train_all = np.load(path + '/labelsPreproc.npy')

X_train, X_val, Y_train, Y_val = train_test_split(X_train_all, Y_train_all,
    test_size=0.25, stratify=np.argmax(Y_train_all, axis=1))

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.4,
    zoom_range=0.3,
    channel_shift_range=0.1
)

# model
model = Sequential()

model.add(Convolution2D(128, (3, 3), padding='same', kernel_initializer='he_normal',
                        activation='relu', input_shape=(img_height, img_width, 1)))
model.add(Convolution2D(128, (3, 3), padding='same', kernel_initializer='he_normal',
                        activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3, 3), padding='same', kernel_initializer='he_normal',
                        activation='relu'))
model.add(Convolution2D(256, (3, 3), padding='same', kernel_initializer='he_normal',
                        activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3, 3), padding='same', kernel_initializer='he_normal',
                        activation='relu'))
model.add(Convolution2D(512, (3, 3), padding='same', kernel_initializer='he_normal',
                        activation='relu'))
model.add(Convolution2D(512, (3, 3), padding='same', kernel_initializer='he_normal',
                        activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes, kernel_initializer='he_normal', activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 128)       1280      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 512)         1180160   
__________

### Train

In [27]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=20,
          validation_data=(X_val, Y_val), verbose=1)

model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['accuracy'])

saveBestModel = ModelCheckpoint('best.kerasModelWeights',
                                monitor='val_acc',
                                verbose=1,
                                save_best_only=True,
                                save_weights_only=True)

history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                              steps_per_epoch=len(X_train) // batch_size,
                              epochs=nb_epoch,
                              validation_data=(X_val, Y_val),
                              callbacks=[saveBestModel],
                              verbose=1)

Train on 4712 samples, validate on 1571 samples
Epoch 1/20


### Predict

In [ ]:
model.load_weights('best.kerasModelWeights')

X_test = np.load(path + 'testPreproc.npy')
Y_test_pred = model.predict_classes(X_test)

vInt2Label = np.vectorize(int2label)
Y_test_pred = vInt2Label(Y_test_pred)

### Display

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

N = range(len(acc))
plt.plot(N, acc, 'bo', label='Training acc')
plt.plot(N, val_acc, 'b', label='Validatoin acc')
plt.title('Training and Validation accuracy')
plt.legend()

plt.figure()

plt.plot(N, loss, 'bo', label='Training loss')
plt.plot(N, val_loss, 'b', label='Validation')
plt.title('Training and validation loss')
plt.legend()

plt.show()
